In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor

from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import cross_validate
import seaborn as sns

In [3]:
data = pd.read_csv("IMDB_netflix4.0.csv")
data

,Unnamed: 0,averageRating,cast,country,date_added,description,duration,genres,isAdult,listed_in,numVotes,rating,release_year,show_id,tconst,title,type,director
0,0,7.8,"Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang ...",South Korea,"November 30, 2019",Brought together by meaningful meals in the pa...,1 Season,"Drama,Romance",0.0,"International TV Shows, Korean TV Shows, Roman...",269.0,TV-14,2019,81193313,tt11214028,Chocolate,TV Show,Hyeong-min Lee
1,1,NaN,Christian Morales,NaN,"November 30, 2019","From Sierra de las Minas to Esquipulas, explor...",67 min,NaN,NaN,"Documentaries, International Movies",NaN,TV-G,2019,81197050,NaN,Guatemala: Heart of the Mayan World,Movie,"Luis Ara, Ignacio Jaunsolo"
2,2,4.3,"Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, ...",India,"November 30, 2019",A goofy copywriter unwittingly convinces the I...,135 min,"Comedy,Drama,Romance",0.0,"Comedies, Dramas, International Movies",1332.0,TV-14,2019,81213894,tt8304386,The Zoya Factor,Movie,Abhishek Sharma
3,3,6.8,"Mama Sane, Amadou Mbow, Ibrahima Traore, Nicol...","France, Senegal, Belgium","November 29, 2019","Arranged to marry a rich man, young Ada is cru...",106 min,Drama,0.0,"Dramas, Independent Movies, International Movies",4567.0,TV-14,2019,81082007,tt10199586,Atlantics,Movie,Mati Diop
4,4,NaN,"Abigail Oliver, Andrea Libman, Briana Buckmast...","Canada, United Kingdom",NaN,"Lovable pug Chip starts kindergarten, makes ne...",2 Seasons,NaN,NaN,Kids' TV,NaN,TV-Y,2019,80213643,NaN,Chip and Potato,TV Show,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5832,5832,NaN,"Jason Faunt, Erin Cahill, Kevin Kleinberg, Deb...",United States,"January 1, 2016",The team launches a new campaign against evil ...,1 Season,NaN,NaN,Kids' TV,NaN,TV-Y7-FV,2001,70213075,NaN,Power Rangers Time Force,TV Show,NaN
5833,5833,NaN,"Ricardo Medina Jr., Alyson Kiperman, Phillip J...","United States, Netherlands, Japan, France","January 1, 2016","Raised in the wilderness, young Cole Evans lea...",1 Season,NaN,NaN,Kids' TV,NaN,TV-Y7-FV,2002,70213073,NaN,Power Rangers Wild Force,TV Show,NaN
5834,5834,NaN,"Mirei Kiritani, Mao Daichi, Wakana Sakai, Masa...",Japan,"December 1, 2015","A young ""fabric geek"" lands a job at an upscal...",1 Season,NaN,NaN,"International TV Shows, TV Dramas",NaN,TV-PG,2015,80067618,NaN,Atelier,TV Show,NaN
5835,5835,NaN,David Attenborough,"United Kingdom, United States, Greece, Italy, ...","April 22, 2015",Discover the glorious variety of life on Earth...,1 Season,NaN,NaN,"British TV Shows, Docuseries, International TV...",NaN,TV-PG,2009,70225722,NaN,Life,TV Show,NaN


# Performance

In [4]:
def performance(var):
    tmp = var.str.split(',', expand=True)
    tmp['rating'] = data['averageRating']
    tmp['votes'] = data['numVotes']
    tmp['overall_performance'] = tmp['rating']*tmp['votes']
    tmp = tmp.loc[tmp.rating.dropna().index]
    tmp = tmp.loc[tmp[0].dropna().index]
    arr = tmp.iloc[:,:-3].values
    arr_2 = arr.reshape(((len(tmp.columns)-3)*len(tmp),1))
    df = pd.DataFrame(arr_2)
    df['rating'] = tmp.rating.repeat((len(tmp.columns)-3)).values
    df['votes'] = tmp.votes.repeat((len(tmp.columns)-3)).values
    df['overall_performance'] = tmp.overall_performance.repeat((len(tmp.columns)-3)).values
    df.dropna(inplace=True)
    df['index'] = np.zeros((len(df)))

    for i in range(len(df)):
        df['index'].iloc[i] = df[0].iloc[i].strip()

    df.drop(columns=[0],inplace=True)

    sum_performance = df.groupby(by = 'index').sum()
    sum_performance['performance'] = sum_performance['overall_performance']/sum_performance['votes']

    
    output = dict(zip(sum_performance.index,sum_performance.performance))
    
    return output

In [5]:
# 计算出director等4指标对应的分数字典

perform_director = performance(data.director)
perform_cast = performance(data.cast)
perform_country = performance(data.country)
perform_genres = performance(data.genres)

# Data Preprocessing

In [10]:
#整理出所需要的部分并去掉空值

movies = data[['show_id', 'title', 'director', 'cast','genres' ,'country','averageRating']].dropna().reset_index(drop = 'True')
movies

,show_id,title,director,cast,genres,country,averageRating
0,81193313,Chocolate,Hyeong-min Lee,"Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang ...","Drama,Romance",South Korea,7.8
1,81213894,The Zoya Factor,Abhishek Sharma,"Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, ...","Comedy,Drama,Romance",India,4.3
2,81082007,Atlantics,Mati Diop,"Mama Sane, Amadou Mbow, Ibrahima Traore, Nicol...",Drama,"France, Senegal, Belgium",6.8
3,81120982,I Lost My Body,Jérémy Clapin,"Hakim Faris, Victoire Du Bois, Patrick d'Assum...","Animation,Drama,Fantasy",France,7.6
4,81227195,Kalushi: The Story of Solomon Mahlangu,Mandla Dube,"Thabo Rametsi, Thabo Malema, Welile Nzuza, Jaf...","Biography,Drama,Thriller",South Africa,6.3
...,...,...,...,...,...,...,...
3281,70028197,Chicken Little,Mark Dindal,"Zach Braff, Joan Cusack, Patrick Stewart, Stev...","Adventure,Animation,Comedy",United States,5.7
3282,80093127,Retribution,Dani de la Torre,"Luis Tosar, Javier Gutiérrez, Elvira Mínguez, ...","Action,Thriller",Spain,6.6
3283,80082739,Fifty,Biyi Bandele,"Dakore Akande, Ireti Doyle, Nse Ikpe-Etim, Omo...","Drama,Romance,Thriller",Nigeria,5.9
3284,70268449,The Square,Jehane Noujaim,"Ahmed Hassan, Khalid Abdalla, Magdy Ashour, Ra...","Documentary,News","United Kingdom, Egypt, United States",8.1


In [11]:
# 将表里的str切割成list

movies.director = [movies.director[i].split(",") for i in  range(np.shape(movies)[0])]
movies.cast = [movies.cast[i].split(",")[0:4] for i in  range(np.shape(movies)[0])]
movies.genres = [movies.genres[i].split(",") for i in  range(np.shape(movies)[0])]
movies.country = [movies.country[i].split(",") for i in  range(np.shape(movies)[0])]

In [12]:
# 整理格式，去掉list里多余的空格

for i in range(np.shape(movies)[0]):
    for j in range(len(movies.director[i])):
        movies.director[i][j] = movies.director[i][j].strip()
    for k in range(len(movies.cast[i])):
        movies.cast[i][k] = movies.cast[i][k].strip()
    for m in range(len(movies.genres[i])):
        movies.genres[i][m] = movies.genres[i][m].strip()
    for n in range(len(movies.country[i])):
        movies.country[i][n] = movies.country[i][n].strip()

In [13]:
movies

,show_id,title,director,cast,genres,country,averageRating
0,81193313,Chocolate,[Hyeong-min Lee],"[Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang...","[Drama, Romance]",[South Korea],7.8
1,81213894,The Zoya Factor,[Abhishek Sharma],"[Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor,...","[Comedy, Drama, Romance]",[India],4.3
2,81082007,Atlantics,[Mati Diop],"[Mama Sane, Amadou Mbow, Ibrahima Traore, Nico...",[Drama],"[France, Senegal, Belgium]",6.8
3,81120982,I Lost My Body,[Jérémy Clapin],"[Hakim Faris, Victoire Du Bois, Patrick d'Assu...","[Animation, Drama, Fantasy]",[France],7.6
4,81227195,Kalushi: The Story of Solomon Mahlangu,[Mandla Dube],"[Thabo Rametsi, Thabo Malema, Welile Nzuza, Ja...","[Biography, Drama, Thriller]",[South Africa],6.3
...,...,...,...,...,...,...,...
3281,70028197,Chicken Little,[Mark Dindal],"[Zach Braff, Joan Cusack, Patrick Stewart, Ste...","[Adventure, Animation, Comedy]",[United States],5.7
3282,80093127,Retribution,[Dani de la Torre],"[Luis Tosar, Javier Gutiérrez, Elvira Mínguez,...","[Action, Thriller]",[Spain],6.6
3283,80082739,Fifty,[Biyi Bandele],"[Dakore Akande, Ireti Doyle, Nse Ikpe-Etim, Om...","[Drama, Romance, Thriller]",[Nigeria],5.9
3284,70268449,The Square,[Jehane Noujaim],"[Ahmed Hassan, Khalid Abdalla, Magdy Ashour, R...","[Documentary, News]","[United Kingdom, Egypt, United States]",8.1


# Calculate the Scores

In [15]:
#计算country得分，无权重

movies['country_score'] = 0.00

for i in range(np.shape(movies)[0]):
    score = 0
    for n in range(len(movies.country[i])):        
        score += perform_country[movies.country[i][n]]
    score = score/(n+1)
    movies['country_score'][i] = score

In [16]:
#计算genres得分，无权重
    
movies['genres_score'] = 0.00

for i in range(np.shape(movies)[0]):
    score = 0
    for n in range(len(movies.genres[i])):        
        score += perform_genres[movies.genres[i][n]]
    score = score/(n+1)
    movies['genres_score'][i] = score

In [17]:
#计算cast得分，有权重
    
movies['cast_score'] = 0.00

for i in range(np.shape(movies)[0]):
    if len(movies.cast[i]) == 1:
        score = perform_cast[movies.cast[i][0]]
    elif len(movies.cast[i]) == 2:
        score = 0.6*perform_cast[movies.cast[i][0]]+ 0.4*perform_cast[movies.cast[i][1]]
    elif len(movies.cast[i]) == 3:
        score = 0.5*perform_cast[movies.cast[i][0]]+ 0.3*perform_cast[movies.cast[i][1]]+ 0.2*perform_cast[movies.cast[i][2]]
    else:
        score = 0.4*perform_cast[movies.cast[i][0]]+ 0.3*perform_cast[movies.cast[i][1]]+ 0.2*perform_cast[movies.cast[i][2]]+ 0.1*perform_cast[movies.cast[i][3]]
    movies['cast_score'][i] = score

In [18]:
#计算director得分，有权重
    
movies['director_score'] = 0.00

for i in range(np.shape(movies)[0]):
    if len(movies.director[i]) == 1:
        score = perform_director[movies.director[i][0]]
    elif len(movies.director[i]) == 2:
        score = 0.6*perform_director[movies.director[i][0]]+ 0.4*perform_director[movies.director[i][1]]
    elif len(movies.director[i]) == 3:
        score = 0.5*perform_director[movies.director[i][0]]+ 0.3*perform_director[movies.director[i][1]]+ 0.2*perform_director[movies.director[i][2]]
    else:
        score = 0.4*perform_director[movies.director[i][0]]+ 0.3*perform_director[movies.director[i][1]]+ 0.2*perform_director[movies.director[i][2]]+ 0.1*perform_director[movies.director[i][3]]
    movies['director_score'][i] = score

In [19]:
movies.head()

,show_id,title,director,cast,genres,country,averageRating,country_score,genres_score,cast_score,director_score
0,81193313,Chocolate,[Hyeong-min Lee],"[Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang...","[Drama, Romance]",[South Korea],7.8,7.152602,7.163802,7.800000,8.177887
1,81213894,The Zoya Factor,[Abhishek Sharma],"[Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor,...","[Comedy, Drama, Romance]",[India],4.3,7.389190,7.034413,6.425421,7.187007
2,81082007,Atlantics,[Mati Diop],"[Mama Sane, Amadou Mbow, Ibrahima Traore, Nico...",[Drama],"[France, Senegal, Belgium]",6.8,6.859206,7.444148,6.800000,6.800000
3,81120982,I Lost My Body,[Jérémy Clapin],"[Hakim Faris, Victoire Du Bois, Patrick d'Assu...","[Animation, Drama, Fantasy]",[France],7.6,6.912322,7.171863,7.459637,7.600000
4,81227195,Kalushi: The Story of Solomon Mahlangu,[Mandla Dube],"[Thabo Rametsi, Thabo Malema, Welile Nzuza, Ja...","[Biography, Drama, Thriller]",[South Africa],6.3,7.507187,7.405523,6.300000,6.300000


# Rating Forecast

In [123]:
# Add interaction terms
df_x = movies[['country_score', 'genres_score','cast_score','director_score']]
# df_x['director_genre']=df_x['director_score']*df_x['genres_score']
# df_x['cast_genre']=df_x['cast_score']*df_x['genres_score']
# df_x['director_cast']=df_x['director_score']*df_x['cast_score']

df_x

,country_score,genres_score,cast_score,director_score
0,7.152602,7.163802,7.800000,8.177887
1,7.389190,7.034413,6.425421,7.187007
2,6.859206,7.444148,6.800000,6.800000
3,6.912322,7.171863,7.459637,7.600000
4,7.507187,7.405523,6.300000,6.300000
...,...,...,...,...
3281,7.137630,7.062404,6.448251,5.700000
3282,6.788469,6.967055,6.789555,6.528677
3283,5.656755,7.098204,5.638840,5.900000
3284,7.052443,7.724632,8.100000,7.386599


In [124]:
df_x = df_x.values
df_y = movies[['averageRating']].values

In [125]:
df_x.shape

(3286, 4)

In [106]:
# a simple ridge regression, just for testing

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


clf = Ridge(alpha=1.0).fit(df_x,df_y)

scores = cross_validate(clf, df_x, df_y, cv=10)
sum(scores['test_score'])/10

0.8709183523641231

## Finding Optimal Parameters

### Model 1 —— Random Forest

In [25]:
# 用GridSearchCV调参，确认最优模型

n_estimators = [10,20,50,100]
min_samples_split = [2, 5, 10, 15]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

param_grid = {'n_estimators': n_estimators,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

 
forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring='neg_mean_squared_error')
 
grid_search.fit(df_x, df_y)

grid_search.best_estimator_


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=15, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [33]:
scores_df = pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score')
scores_df = scores_df[['param_bootstrap','param_min_samples_leaf','param_min_samples_split',
                      'param_n_estimators','mean_test_score','rank_test_score']]
scores_df.head()

,param_bootstrap,param_min_samples_leaf,param_min_samples_split,param_n_estimators,mean_test_score,rank_test_score
47,True,4,15,100,-0.174505,1
46,True,4,15,50,-0.175341,2
39,True,4,5,100,-0.175701,3
34,True,4,2,50,-0.175716,4
43,True,4,10,100,-0.176143,5


### Model 2 —— CART

In [34]:
min_samples_split = [5, 10, 15, 20]
min_samples_leaf = [1, 2, 4]
max_depth = [2,3,4,5,6,7]

param_grid = { 'max_depth':max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

cart = DecisionTreeRegressor()

grid_search = GridSearchCV(cart, param_grid, cv=5,
                          scoring='neg_mean_squared_error')
 
grid_search.fit(df_x, df_y)

grid_search.best_estimator_

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=7,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=20,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [36]:
scores_df = pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score')
scores_df = scores_df[['param_min_samples_leaf','param_min_samples_split',
                      'param_max_depth','mean_test_score','rank_test_score']]
scores_df.head()

,param_min_samples_leaf,param_min_samples_split,param_max_depth,mean_test_score,rank_test_score
71,4,20,7,-0.199991,1
70,4,15,7,-0.200353,2
58,4,15,6,-0.200600,3
59,4,20,6,-0.200656,4
57,4,10,6,-0.202520,5


### Model 3 —— KNN

In [37]:
n_neighbors = range(1,10)
leaf_size = [10, 20, 30, 50]

param_grid = { 'n_neighbors':n_neighbors,
               'leaf_size':leaf_size}

knn = KNeighborsRegressor()

grid_search = GridSearchCV(knn, param_grid, cv=5,
                          scoring='neg_mean_squared_error')
 
grid_search.fit(df_x, df_y)

grid_search.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=10, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                    weights='uniform')

In [38]:
scores_df = pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score')
scores_df = scores_df[['param_n_neighbors','param_leaf_size',
                      'mean_test_score','rank_test_score']]
scores_df.head()

,param_n_neighbors,param_leaf_size,mean_test_score,rank_test_score
8,9,10,-0.182700,1
17,9,20,-0.182740,2
35,9,50,-0.182749,3
26,9,30,-0.182759,4
24,7,30,-0.185121,5


## Model Comparison

In [126]:
model1 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

model2 = DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=15, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

model3 = KNeighborsRegressor(algorithm='auto', leaf_size=50, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                    weights='uniform') 

from sklearn.model_selection import KFold
kf = KFold(10,shuffle=True,random_state=13)
cv_results1=[]
cv_results2=[]
cv_results3=[]
for train,test in kf.split(df_x):
    X_train = df_x[train]
    y_train = df_y[train]
    val_X = df_x[test]
    val_y = df_y[test]
    
    cv_results1.append(model1.fit(X_train,y_train).score(val_X,val_y))
    cv_results2.append(model2.fit(X_train,y_train).score(val_X,val_y))
    cv_results3.append(model1.fit(X_train,y_train).score(val_X,val_y))

test_score = np.array([cv_results1,cv_results2,cv_results3]).transpose()
comparison = pd.DataFrame(test_score)
comparison.columns = ['Random Forest', 'CART', 'KNN']

In [127]:
comparison

,Random Forest,CART,KNN
0,0.880400,0.859895,0.881904
1,0.852354,0.844972,0.852466
2,0.893653,0.882913,0.896701
3,0.906742,0.888483,0.907379
4,0.851239,0.829868,0.854377
5,0.896189,0.884348,0.894607
6,0.907640,0.891614,0.908280
7,0.878507,0.853262,0.877030
8,0.903310,0.885441,0.904996
9,0.864348,0.863129,0.864386


In [128]:
comparison.describe()

,Random Forest,CART,KNN
count,10.000000,10.000000,10.000000
mean,0.883438,0.868392,0.884212
std,0.021581,0.021241,0.021499
min,0.851239,0.829868,0.852466
25%,0.867888,0.854920,0.867547
50%,0.887026,0.873021,0.888255
75%,0.901530,0.885168,0.902922
max,0.907640,0.891614,0.908280


## Prediction

In [25]:
model1.fit(df_x, df_y)
y_predict =  model1.predict(df_x)
movies['predictRating']=y_predict

In [27]:
movies[['title', 'averageRating', 'predictRating']]

,title,averageRating,predictRating
0,Chocolate,7.8,7.873713
1,The Zoya Factor,4.3,6.130301
2,Atlantics,6.8,6.811612
3,I Lost My Body,7.6,7.618004
4,Kalushi: The Story of Solomon Mahlangu,6.3,6.276326
...,...,...,...
3281,Chicken Little,5.7,5.712703
3282,Retribution,6.6,6.512923
3283,Fifty,5.9,5.783519
3284,The Square,8.1,8.151541
